In [26]:
import numpy as np
import os

class SimpleDatasetLoader:
    def init(self):
        return

    def load(self, imagePaths, verbose=-1):
        # initialize the list of features and labels
        data = []
        labels = []
        # loop over the input images
        for (i, imagePath) in enumerate(imagePaths):
            # load the image and extract the class label assuming
            # that our path has the following format:
            # /path/to/dataset/{class}/{image}.jpg
            image = cv2.imread(imagePath)
            label = f'{imagePath.split(os.path.sep)[-3]}_{imagePath.split(os.path.sep)[-2]}'

            data.append(image)
            labels.append(label)

            # show an update every verbose images
            if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
                print("[INFO] processed {}/{}".format(i + 1,
                                                      len(imagePaths)))
                # return a tuple of the data and labels
        return (np.array(data), np.array(labels))


In [49]:
import cv2
import json
import pandas as pd
from imutils import paths
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
    
class Parametros:
  dataset = "./images"
  neighbors = 1
  jobs = -1

args = vars(Parametros)

print("[INFO] loading images...")
imagePaths = list(paths.list_images(args["dataset"]))
sdl = SimpleDatasetLoader()
(data, labels) = sdl.load(imagePaths, verbose=500)

sift_df = pd.DataFrame()

for i in range(len(data)):
    image = data[i]
    label = labels[i]

    # GENERATING :SIFT: METRICS
    sift = cv2.SIFT_create()

    keypoints, descriptors = sift.detectAndCompute(image, None)
    author = label

    # CONVERTING KEYPOINTS TO COORDINATES
    keypoint_list = list(keypoints)
    descriptors_list = descriptors[np.random.randint(descriptors.shape[0], size=100)].tolist()


    cols = list(range(len(descriptors_list[0])))
    cols.insert(0, 'class_name')

    for descriptors in descriptors_list: 
        descriptors.insert(0, author)

    sift_matrix = list(descriptors_list)

    df = pd.DataFrame(sift_matrix, columns=cols)

    sift_df = pd.concat([sift_df, df])

[INFO] loading images...


In [50]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

X = sift_df.loc[:, 0:].values
y = sift_df[['class_name']].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.181, random_state = 42)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

C:\Users\castr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.6895833333333333

In [51]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pickle

knn = pickle.load(open("../SIFT_histograma_knn/ml_models/LeitorHistograma_v2.pkl", "rb"))


print(classification_report(y_test, y_pred, target_names=sift_df.class_name.unique()))
ConfusionMatrixDisplay.from_estimator(
    knn, X_test, y_test, display_labels=sift_df.class_name.unique(), xticks_rotation="vertical"
)
plt.tight_layout()
plt.show()

                           precision    recall  f1-score   support

      monet_ai_signatures       0.61      0.58      0.60       380
monet_original_signatures       0.73      0.76      0.75       580

                 accuracy                           0.69       960
                macro avg       0.67      0.67      0.67       960
             weighted avg       0.69      0.69      0.69       960



C:\Users\castr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.3.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


ValueError: X has 128 features, but KNeighborsClassifier is expecting 50 features as input.